<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Entrada-Antes-das-8h" data-toc-modified-id="Entrada-Antes-das-8h-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Entrada Antes das 8h</a></span></li><li><span><a href="#Ajustes-Manuais" data-toc-modified-id="Ajustes-Manuais-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Ajustes Manuais</a></span></li><li><span><a href="#Intervalo-no-Início-ou-Final-da-Jornada" data-toc-modified-id="Intervalo-no-Início-ou-Final-da-Jornada-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Intervalo no Início ou Final da Jornada</a></span></li></ul></div>

In [1]:
# Importando bibliotecas
import pandas as pd
import os
import time
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import shutil
from warnings import filterwarnings
filterwarnings('ignore')

In [49]:
def time_to_datetime(df, date_col, time_col, prefix='ponto_', sufix='', drop=False):
    """
    Aplicação
    ----------

    
    Parâmetros
    ----------

    
    Retorno
    ----------

    
    Dependências
    ----------

    """
    
    # Cria coluna com concatenação entre data e tempo
    try:
        df[prefix + time_col + sufix] = df[date_col] + ' ' + df[time_col]
    except TypeError:
        # Uma ou mais colunas (date_col e time_col) não são strings
        df[prefix + time_col + sufix] = df[date_col].astype(str) + ' ' + df[time_col].astype(str)
    
    # Transforma em datetime
    df[prefix + time_col + sufix] = pd.to_datetime(df[prefix + time_col + sufix])
    
    # Valida drop da coluna antiga
    if drop & (prefix != '') & (sufix != ''):
        return df.drop(time_col, axis=1)
    else:
        return df

In [40]:
def valida_dados_origem(path, filename, data_mod=True, verbose=False):
    """
    Aplicação
    ----------
    Função para verificar se um determinado arquivo encontra-se em um determinado diretório origem.
    Como parâmetro adicional, pode-se validar a atualização do mesmo a partir do mês de modificação.
    
    Parâmetros
    ----------
    path: diretório completo da origem na qual será realizada a busca e validação
    filename: nome do arquivo (incluindo extensão) a ser procurado e validado na origem
    
    Retorno
    bool: variável booleana contendo validação de presença e data de modificação do arquivo
    
    Dependências
    bibliotecas: os, time, datetime
    """
    
    # Valida existência de arquivo na origem
    files_in_origin = os.listdir(path)
    if filename in files_in_origin:
        
        # Retorna parâmetros de modificação do arquivo
        mod_date = os.path.getmtime(path + '/' + filename)
        anomes_file = time.strftime('%Y%m', time.localtime(mod_date))
        anomes_atu = datetime.now().strftime('%Y%m')
        
        # Valida data de modificação com mês de atualização
        if anomes_atu == anomes_file:
            if verbose:
                print(f'Arquivo presente na origem e com data atualizada!')
            return True
        else:
            if verbose:
                print(f'[ALERTA] Arquivo presente na origem, porém destualizado! Leitura interrompida.')
                print(f'\nMês atual: {anomes_atu:>19}\nAtualização do arquivo: {anomes_file}')
            return False
    else:
        # Arquivo não presente na origem
        if verbose:
            print(f'[ALERTA] Arquivo {filename} não presente na origem ({path})')
            print(f'\nConteúdo do diretório: \n{files_in_origin}')
        return False

In [41]:
def copia_arquivo(origem, destino, verbose=False):
    """
    Aplicação
    ----------
    Copia um arquivo dos sistema operacional para um determinado destino
    
    Parâmetros
    ----------
    origem: caminho completo da origem + filename com extensão do arquivo a ser copiado
    destsino: caminho completo do destino + filename com extensão do arquivo copiado
    
    Retorno
    ----------
    None
    
    Dependências
    ----------
    bibliotecas: shutil
    """
    
    shutil.copyfile(origem, destino)

In [51]:
# Inicializando variáveis de controle
src_path = r'D:\Users\thiagoPanini\github_files\programming_languages\bank-stuff\gt-trab\data\raw_origin'
dst_path = r'D:\Users\thiagoPanini\github_files\programming_languages\bank-stuff\gt-trab\data'
ponto_filename = 'RANDOM_PONTO.csv'
src_datapath = os.path.join(src_path, ponto_filename)
dst_datapath = os.path.join(dst_path, ponto_filename)

# Validando arquivo de ponto presente na origem
if valida_dados_origem(path=src_path, filename=ponto_filename):
    
    # Copia arquivo da origem ao destino do projeto
    copia_arquivo(origem=src_datapath, destino=dst_datapath)
    
# Lendo dados
raw_ponto = pd.read_csv(dst_datapath)
print(f'Volumetria base original: {raw_ponto.shape}')

# Filtrando mês atual
anomes_atu = int((datetime.now() - relativedelta(months=0)).strftime('%Y%m'))
df_ponto = raw_ponto.copy()
df_ponto = df_ponto.query('mesref == @anomes_atu')
print(f'\nVolumetria base filtrada mês atual: {df_ponto.shape}')
df_ponto.sort_values(by=['numfunc', 'dia'], inplace=True)
df_ponto.head()

Volumetria base original: (100000, 12)

Volumetria base filtrada mês atual: (49652, 12)


,mesref,numfunc,racf,dia,dia_baliza,hora,desc_tipo_marcacao,desc_tipo_registro,deptid,database,dt_venc_acordo,codigo
2225,202008,1,ABCDEF,2020-08-01,2020-08-01,09:53,Saída,Incluído,803550957,2020-08-21,2020-08-21,TP1021
7698,202008,1,ABCDEF,2020-08-01,2020-08-01,10:51,Entrada,Original,189130830,2020-08-21,2020-08-21,TP1021
8337,202008,1,ABCDEF,2020-08-01,2020-08-01,17:03,Entrada,Original,400314106,2020-08-21,2020-08-21,TP1021
8535,202008,1,ABCDEF,2020-08-01,2020-08-01,14:58,Saída,Original,364202007,2020-08-21,2020-08-21,TP1021
10148,202008,1,ABCDEF,2020-08-01,2020-08-01,13:07,Saída,Incluído,754475693,2020-08-21,2020-08-21,TP1021


In [52]:
# Tipos primitivos originais
df_ponto.dtypes

mesref                 int64
numfunc                int64
racf                  object
dia                   object
dia_baliza            object
hora                  object
desc_tipo_marcacao    object
desc_tipo_registro    object
deptid                 int64
database              object
dt_venc_acordo        object
codigo                object
dtype: object

In [53]:
# Transformando tipos primitivos
date_cols = ['dia', 'dia_baliza', 'database', 'dt_venc_acordo']
for col in date_cols:
    df_ponto[col] = pd.to_datetime(df_ponto[col])

# Convertendo coluna de hora
df_ponto = time_to_datetime(df=df_ponto, date_col='dia', time_col='hora', 
                            prefix='', sufix='_ponto', drop=True)

# Novos tipos primitivos
df_ponto.dtypes

mesref                         int64
numfunc                        int64
racf                          object
dia                   datetime64[ns]
dia_baliza            datetime64[ns]
hora                          object
desc_tipo_marcacao            object
desc_tipo_registro            object
deptid                         int64
database              datetime64[ns]
dt_venc_acordo        datetime64[ns]
codigo                        object
hora_ponto            datetime64[ns]
dtype: object

In [54]:
df_ponto.head()

,mesref,numfunc,racf,dia,dia_baliza,hora,desc_tipo_marcacao,desc_tipo_registro,deptid,database,dt_venc_acordo,codigo,hora_ponto
2225,202008,1,ABCDEF,2020-08-01,2020-08-01,09:53,Saída,Incluído,803550957,2020-08-21,2020-08-21,TP1021,2020-08-01 09:53:00
7698,202008,1,ABCDEF,2020-08-01,2020-08-01,10:51,Entrada,Original,189130830,2020-08-21,2020-08-21,TP1021,2020-08-01 10:51:00
8337,202008,1,ABCDEF,2020-08-01,2020-08-01,17:03,Entrada,Original,400314106,2020-08-21,2020-08-21,TP1021,2020-08-01 17:03:00
8535,202008,1,ABCDEF,2020-08-01,2020-08-01,14:58,Saída,Original,364202007,2020-08-21,2020-08-21,TP1021,2020-08-01 14:58:00
10148,202008,1,ABCDEF,2020-08-01,2020-08-01,13:07,Saída,Incluído,754475693,2020-08-21,2020-08-21,TP1021,2020-08-01 13:07:00


In [58]:
# Definindo colunas a serem filtradas para armazenamento dos dados
cols_indicadores = ['numfunc', 'data', 'hora', 'tipo_marcacao', 'tipo_registro', 'tipo_ocorrencia']
cols_ponto = ['numfunc', 'dia', 'hora_ponto', 'desc_tipo_marcacao', 'desc_tipo_registro', 'tipo_ocorrencia']

# Criando DataFrame de indicadores para armazenar funcionais e ocorrências
df_indicadores = pd.DataFrame(columns=cols_indicadores)
df_indicadores

,numfunc,data,hora,tipo_marcacao,tipo_registro,tipo_ocorrencia


### Entrada Antes das 8h

_Escopo:_ Recorrência de marcações de entrada antes das 8h em um mesmo mês

* **Origens:** PONTO
* **Atributos:** `numfunc`, `hora`, `desc_tipo_marcacao`.
* **Procedimento:**
    - Criar flag para marcações de entrada (`desc_tipo_marcacao == 'Entrada'`)
    - Criar flag para marcações antes das 8h (`hora.hour < 8`)
    - Multiplicar os flags para filtrar entradas antes das 8h

In [59]:
def ind_entrada_pre_8h(df_indicadores, cols_ponto=cols_ponto, cols_indicadores=cols_indicadores):
    """
    Aplicação
    ----------

    
    Parâmetros
    ----------

    
    Retorno
    ----------

    
    Dependências
    ----------

    """
    
    # Criando marcação para o indicador
    mask = (df_ponto['desc_tipo_marcacao'] == 'Entrada') & df_ponto['hora_ponto'].apply(lambda x: x.hour < 8)
    df_ponto['entrada_pre_8h'] = mask * 1
    
    # Criando base consolidada de indicadores
    temp_indicador = df_ponto[mask]
    temp_indicador['tipo_ocorrencia'] = 'Entrada antes das 8h'
    temp_indicador = temp_indicador.loc[:, cols_ponto]
    temp_indicador.columns = cols_indicadores
    
    # Append na base de indicadores
    df_indicadores = df_indicadores.append(temp_indicador)
    
    return df_ponto, df_indicadores

In [60]:
# Executando função
df_indicadores = pd.DataFrame(columns=cols_indicadores)
df_ponto, df_indicadores = ind_entrada_pre_8h(df_indicadores)

# Verificando volumetrias
print(f'Volumetria da base de marcação de ponto: {df_ponto.shape}')
print(f'Volumetria da base de indicadores: {df_indicadores.shape}')
df_indicadores.head()

Volumetria da base de marcação de ponto: (49652, 14)
Volumetria da base de indicadores: (2129, 6)


,numfunc,data,hora,tipo_marcacao,tipo_registro,tipo_ocorrencia
36015,1,2020-08-01,2020-08-01 07:47:00,Entrada,Original,Entrada antes das 8h
35251,1,2020-08-02,2020-08-02 07:12:00,Entrada,Incluído,Entrada antes das 8h
92746,1,2020-08-02,2020-08-02 07:58:00,Entrada,Incluído,Entrada antes das 8h
45728,1,2020-08-04,2020-08-04 07:59:00,Entrada,Original,Entrada antes das 8h
79628,1,2020-08-06,2020-08-06 07:48:00,Entrada,Original,Entrada antes das 8h


### Ajustes Manuais

_Escopo:_ Recorrência de marcações incluídas em um mês

* **Origens:** PONTO
* **Atributos:** `numfunc`, `hora`, `desc_tipo_registro`.
* **Procedimento:**
     - Filtrar marcações de entrada na base origem (`desc_tipo_registro == 'Incluído'`)
     - Agrupar dados por numfunc e dia

In [10]:
def ind_ajustes_manuais(df_indicadores, cols_ponto=cols_ponto, cols_indicadores=cols_indicadores):
    """
    Aplicação
    ----------

    
    Parâmetros
    ----------

    
    Retorno
    ----------

    
    Dependências
    ----------

    """
    
    # Criando marcação para o indicador
    mask = df_ponto['desc_tipo_registro'] == 'Incluído'
    df_ponto['ajuste_manual'] = mask * 1
    
    # Criando base consolidada de indicadores
    temp_indicador = df_ponto[mask]
    temp_indicador['tipo_ocorrencia'] = 'Ajuste manual'
    temp_indicador = temp_indicador.loc[:, cols_ponto]
    temp_indicador.columns = cols_indicadores
    
    # Append na base de indicadores
    df_indicadores = df_indicadores.append(temp_indicador)
    
    return df_ponto, df_indicadores

In [11]:
# Executando função
df_ponto, df_indicadores = ind_ajustes_manuais(df_indicadores)

# Verificando volumetrias
print(f'Volumetria da base de marcação de ponto: {df_ponto.shape}')
print(f'Volumetria da base de indicadores: {df_indicadores.shape}')

Volumetria da base de marcação de ponto: (49652, 14)
Volumetria da base de indicadores: (26998, 6)


In [12]:
# Verificando base de ponto
df_ponto[(df_ponto['ajuste_manual'] == 1) & (df_ponto['entrada_pre_8h'] == 1)].head()

,mesref,numfunc,racf,dia,dia_baliza,desc_tipo_marcacao,desc_tipo_registro,deptid,database,dt_venc_acordo,codigo,hora_custom,entrada_pre_8h,ajuste_manual
35251,202008,1,ABCDEF,2020-08-02,2020-08-02,Entrada,Incluído,663215648,2020-08-21,2020-08-21,TP1021,07:12:00,1,1
92746,202008,1,ABCDEF,2020-08-02,2020-08-02,Entrada,Incluído,827972756,2020-08-21,2020-08-21,TP1021,07:58:00,1,1
815,202008,1,ABCDEF,2020-08-08,2020-08-08,Entrada,Incluído,648285837,2020-08-21,2020-08-21,TP1021,07:21:00,1,1
97460,202008,1,ABCDEF,2020-08-08,2020-08-08,Entrada,Incluído,247038855,2020-08-21,2020-08-21,TP1021,07:51:00,1,1
83875,202008,1,ABCDEF,2020-08-09,2020-08-09,Entrada,Incluído,121479489,2020-08-21,2020-08-21,TP1021,07:38:00,1,1


In [14]:
# Verificando base de indicadores para 1 funcional em 1 dia
df_indicadores[(df_indicadores['numfunc'] == 1) & (df_indicadores['data'] == '2020-08-02')]

,numfunc,data,hora,tipo_marcacao,tipo_registro,tipo_ocorrencia
35251,1,2020-08-02,07:12:00,Entrada,Incluído,Entrada antes das 8h
92746,1,2020-08-02,07:58:00,Entrada,Incluído,Entrada antes das 8h
17396,1,2020-08-02,18:01:00,Entrada,Incluído,Ajuste manual
27412,1,2020-08-02,09:30:00,Entrada,Incluído,Ajuste manual
35251,1,2020-08-02,07:12:00,Entrada,Incluído,Ajuste manual
56853,1,2020-08-02,14:47:00,Entrada,Incluído,Ajuste manual
67018,1,2020-08-02,08:13:00,Entrada,Incluído,Ajuste manual
71864,1,2020-08-02,16:34:00,Saída,Incluído,Ajuste manual
75645,1,2020-08-02,14:23:00,Saída,Incluído,Ajuste manual
92746,1,2020-08-02,07:58:00,Entrada,Incluído,Ajuste manual


### Intervalo no Início ou Final da Jornada

_Escopo:_ Recorrência de marcações de intervalo no início ou no final da jornada.

* **Origens:** PONTO
* **Atributos:** `numfunc`, `hora`, `desc_tipo_marcacao`.
* **Procedimento:**
     - Adicionar um atributo na base considerando marcações "Expediente" e "Intervalo"
     - Medir diferença entre marcações "Intervalo" com marcações "Expediente"
     - Marcar ponto de marcações realizadas em intervalos curtos de tempo entre "Intervalo" e "Expediente"
* **Observações:**
    - Se marcações de intervalo ocorrerem entre 11:30h e 14:00h, a regra NÃO deve ser aplicada
    - Cargos comissionados (480 min no dia) tem limite de 2h de marcações para intervalo
    - Cargos não-comissionados (360 min no dia) tem limite de 1h de marcações para intervalo

**PROBLEMA: TEM QUE AGRUPAR POR DIA PRA VER AS HORAS MÍNIMA E MÁXIMA EM CADA DIA**

In [77]:
# Amostrando duas funcionais da base e criando coluna de limite de comissionados
sample = df_ponto.query('numfunc in (1, 2)').sort_values(by=['numfunc', 'dia', 'hora_custom'])
sample = sample[['numfunc', 'dia', 'hora_custom', 'desc_tipo_marcacao', 'desc_tipo_registro']]
comiss = {1: 7200, 2: 3600} # Cargos comissionados e não-comissionados
sample['dif_limite'] = sample['numfunc'].map(comiss)
sample.head()

,numfunc,dia,hora_custom,desc_tipo_marcacao,desc_tipo_registro,dif_limite
82952,1,2020-08-01,07:27:00,Saída,Incluído,7200
36015,1,2020-08-01,07:47:00,Entrada,Original,7200
18573,1,2020-08-01,09:18:00,Entrada,Incluído,7200
2225,1,2020-08-01,09:53:00,Saída,Incluído,7200
42002,1,2020-08-01,10:15:00,Saída,Incluído,7200


In [15]:
# Amostrando duas funcionais da base e criando coluna de limite de comissionados
sample = df_ponto.query('numfunc in (1, 2)').sort_values(by=['numfunc', 'dia', 'hora_custom'])
sample = sample[['numfunc', 'dia', 'hora_custom', 'desc_tipo_marcacao', 'desc_tipo_registro']]
comiss = {1: 7200, 2: 3600} # Cargos comissionados e não-comissionados
sample['dif_limite'] = sample['numfunc'].map(comiss)

# Agrupando por hora mínima de entrada e hora máxima de saída
sample_entrada = sample.query('desc_tipo_marcacao == "Entrada"')
sample_saida = sample.query('desc_tipo_marcacao == "Saída"')

# Gerando bases de primiera entrada e última saída
hora_prim_entrada = sample_entrada.groupby(by=['numfunc', 'dia'], as_index=False).min().loc[:, ['numfunc', 'dia', 'hora_custom']]
hora_prim_entrada.columns = ['numfunc', 'dia', 'hora_prim_entrada']
hora_ult_saida = sample_saida.groupby(by=['numfunc', 'dia'], as_index=False).max().loc[:, ['numfunc', 'dia', 'hora_custom']]
hora_ult_saida.columns = ['numfunc', 'dia', 'hora_ult_saida']

# Cruzando dados
sample = sample.merge(hora_prim_entrada, how='left', on=['numfunc', 'dia'])
sample = sample.merge(hora_ult_saida, how='left', on=['numfunc', 'dia'])

# Marcações de intervalo (hora not in (primeira_entrada, ultima_saida))
mask_intervalo = ~((sample['hora_custom'] == sample['hora_prim_entrada']) | (sample['hora_custom'] == sample['hora_ult_saida']))
sample['flag_intervalo'] = mask_intervalo * 1
sample.head()

,numfunc,dia,hora_custom,desc_tipo_marcacao,desc_tipo_registro,dif_limite,hora_prim_entrada,hora_ult_saida,flag_intervalo
0,1,2020-08-01,07:27:00,Saída,Incluído,7200,07:47:00,18:38:00,1
1,1,2020-08-01,07:47:00,Entrada,Original,7200,07:47:00,18:38:00,0
2,1,2020-08-01,09:18:00,Entrada,Incluído,7200,07:47:00,18:38:00,1
3,1,2020-08-01,09:53:00,Saída,Incluído,7200,07:47:00,18:38:00,1
4,1,2020-08-01,10:15:00,Saída,Incluído,7200,07:47:00,18:38:00,1


In [26]:
sample['hora'] = sample['dia'].astype(str) + ' ' + sample['hora_custom'].astype(str)
sample['hora'] = pd.to_datetime(sample['hora'])
sample.head()

,numfunc,dia,hora_custom,desc_tipo_marcacao,desc_tipo_registro,dif_limite,hora_prim_entrada,hora_ult_saida,flag_intervalo,hora
0,1,2020-08-01,07:27:00,Saída,Incluído,7200,07:47:00,18:38:00,1,2020-08-01 07:27:00
1,1,2020-08-01,07:47:00,Entrada,Original,7200,07:47:00,18:38:00,0,2020-08-01 07:47:00
2,1,2020-08-01,09:18:00,Entrada,Incluído,7200,07:47:00,18:38:00,1,2020-08-01 09:18:00
3,1,2020-08-01,09:53:00,Saída,Incluído,7200,07:47:00,18:38:00,1,2020-08-01 09:53:00
4,1,2020-08-01,10:15:00,Saída,Incluído,7200,07:47:00,18:38:00,1,2020-08-01 10:15:00


In [22]:
sample.head()

,numfunc,dia,hora_custom,desc_tipo_marcacao,desc_tipo_registro,dif_limite,hora_prim_entrada,hora_ult_saida,flag_intervalo,hora
0,1,2020-08-01,07:27:00,Saída,Incluído,7200,07:47:00,18:38:00,1,2020-08-24 07:27:00
1,1,2020-08-01,07:47:00,Entrada,Original,7200,07:47:00,18:38:00,0,2020-08-24 07:47:00
2,1,2020-08-01,09:18:00,Entrada,Incluído,7200,07:47:00,18:38:00,1,2020-08-24 09:18:00
3,1,2020-08-01,09:53:00,Saída,Incluído,7200,07:47:00,18:38:00,1,2020-08-24 09:53:00
4,1,2020-08-01,10:15:00,Saída,Incluído,7200,07:47:00,18:38:00,1,2020-08-24 10:15:00


In [17]:
mask_regra_intervalo = (sample['hora_custom'] > datetime.strptime('11:30:00', '%H:%M:%S'))

TypeError: '>' not supported between instances of 'datetime.time' and 'datetime.datetime'

In [41]:
# Filtrando marcações de intervalo
sample_intervalo = sample.query('flag_intervalo == "Intervalo"')
sample_intervalo_entrada = sample_intervalo.query('desc_tipo_marcacao == "Entrada"')
sample_intervalo_saida = sample_intervalo.query('desc_tipo_marcacao == "Saída"')

# Gerando bases de primeira saída e última entrada
hora_prim_saida = sample_intervalo_saida.groupby(by=['numfunc', 'dia'], 
                                                 as_index=False).min().loc[:, ['numfunc', 'dia', 'hora_custom']]
hora_prim_saida.columns = ['numfunc', 'dia', 'hora_prim_saida']
hora_ult_entrada = sample_intervalo_entrada.groupby(by=['numfunc', 'dia'], 
                                                    as_index=False).max().loc[:, ['numfunc', 'dia', 'hora_custom']]
hora_ult_entrada.columns = ['numfunc', 'dia', 'hora_ult_entrada']

# Cruzando dados
sample = sample.merge(hora_prim_saida, how='left', on=['numfunc', 'dia'])
sample = sample.merge(hora_ult_entrada, how='left', on=['numfunc', 'dia'])
sample.head()

,numfunc,dia,hora_custom,desc_tipo_marcacao,desc_tipo_registro,dif_limite,hora_prim_entrada,hora_ult_saida,flag_intervalo,hora_prim_saida,hora_ult_entrada
0,9,2020-08-01,07:37:00,Saída,Incluído,7200,11:09:00,18:48:00,Intervalo,07:37:00,18:01:00
1,9,2020-08-01,08:16:00,Saída,Incluído,7200,11:09:00,18:48:00,Intervalo,07:37:00,18:01:00
2,9,2020-08-01,09:51:00,Saída,Original,7200,11:09:00,18:48:00,Intervalo,07:37:00,18:01:00
3,9,2020-08-01,11:09:00,Entrada,Original,7200,11:09:00,18:48:00,Expediente,07:37:00,18:01:00
4,9,2020-08-01,11:41:00,Entrada,Incluído,7200,11:09:00,18:48:00,Intervalo,07:37:00,18:01:00


In [45]:
def sub_datetimes(h1, h2):
    return datetime.combine(date.today(), h1) - datetime.combine(date.today(), h2)

sample['delta_prim_saida'] = sample.apply(lambda row: sub_datetimes(row['hora_prim_entrada'], row['hora_prim_saida']), axis=1)

TypeError: ('combine() argument 2 must be datetime.time, not float', 'occurred at index 210')

In [53]:
sample.head(215)

,numfunc,dia,hora_custom,desc_tipo_marcacao,desc_tipo_registro,dif_limite,hora_prim_entrada,hora_ult_saida,flag_intervalo,hora_prim_saida,hora_ult_entrada
0,9,2020-08-01,07:37:00,Saída,Incluído,7200,11:09:00,18:48:00,Intervalo,07:37:00,18:01:00
1,9,2020-08-01,08:16:00,Saída,Incluído,7200,11:09:00,18:48:00,Intervalo,07:37:00,18:01:00
2,9,2020-08-01,09:51:00,Saída,Original,7200,11:09:00,18:48:00,Intervalo,07:37:00,18:01:00
3,9,2020-08-01,11:09:00,Entrada,Original,7200,11:09:00,18:48:00,Expediente,07:37:00,18:01:00
4,9,2020-08-01,11:41:00,Entrada,Incluído,7200,11:09:00,18:48:00,Intervalo,07:37:00,18:01:00
5,9,2020-08-01,12:05:00,Entrada,Incluído,7200,11:09:00,18:48:00,Intervalo,07:37:00,18:01:00
6,9,2020-08-01,12:10:00,Saída,Incluído,7200,11:09:00,18:48:00,Intervalo,07:37:00,18:01:00
7,9,2020-08-01,12:36:00,Entrada,Incluído,7200,11:09:00,18:48:00,Intervalo,07:37:00,18:01:00
8,9,2020-08-01,13:32:00,Saída,Incluído,7200,11:09:00,18:48:00,Intervalo,07:37:00,18:01:00
9,9,2020-08-01,14:14:00,Entrada,Incluído,7200,11:09:00,18:48:00,Intervalo,07:37:00,18:01:00


In [42]:
sample['dif_primeira_saida'] = sub_datetimes(sample['hora_prim_entrada'], sample['hora_prim_saida'])

TypeError: combine() argument 2 must be datetime.time, not Series

In [23]:
# Amostrando duas funcionais da base
sample = df_ponto.query('numfunc in (9, 10)').sort_values(by=['numfunc', 'dia', 'hora_custom'])
sample = sample[['numfunc', 'dia', 'hora_custom', 'desc_tipo_marcacao', 'desc_tipo_registro']]
comiss = {9: 7200, 10: 3600}
sample['dif_limite'] = sample['numfunc'].map(comiss)

# Agrupando por hora mínima de entrada e hora máxima de saída
sample_entrada = sample.query('desc_tipo_marcacao == "Entrada"')
sample_saida = sample.query('desc_tipo_marcacao == "Saída"')

# Gerando bases de primiera entrada e última saída
hora_prim_entrada = sample_entrada.groupby(by=['numfunc', 'dia'], as_index=False).min().loc[:, ['numfunc', 'dia', 'hora_custom']]
hora_prim_entrada.columns = ['numfunc', 'dia', 'hora_prim_entrada']
hora_ult_saida = sample_saida.groupby(by=['numfunc', 'dia'], as_index=False).max().loc[:, ['numfunc', 'dia', 'hora_custom']]
hora_ult_saida.columns = ['numfunc', 'dia', 'hora_ult_saida']

# Cruzando dados
sample = sample.merge(hora_prim_entrada, how='left', on=['numfunc', 'dia'])
sample = sample.merge(hora_ult_saida, how='left', on=['numfunc', 'dia'])

# Transformando dados
sample['hora_prim_entrada'] = sample['hora_prim_entrada'].apply(lambda x: datetime.combine(date.today(), x))
sample['hora_ult_saida'] = sample['hora_ult_saida'].apply(lambda x: datetime.combine(date.today(), x))
sample['hora_custom'] = sample['hora_custom'].apply(lambda x: datetime.combine(date.today(), x))

# Validando marcações de intervalo e expediente
sample['flag_intervalo'] = (sample['hora_custom'] == sample['hora_prim_entrada']) | (sample['hora_custom'] == sample['hora_ult_saida'])
sample['flag_intervalo'] = sample['flag_intervalo'].apply(lambda x: 'Expediente' if x else 'Intervalo')
sample.head()

NameError: name 'date' is not defined

In [ ]:
# Agrupando marcações relacionadas a intervalo
df_intervalo = sample.query('flag_intervalo == "Intervalo"')
intervalo_entrada = df_intervalo.query('desc_tipo_marcacao == "Entrada"')
intervalo_saida = df_intervalo.query('desc_tipo_marcacao == "Saída"')

# Gerando bases de primiera entrada e última saída
hora_prim_saida = intervalo_saida.groupby(by=['numfunc', 'dia'], as_index=False).min().loc[:, ['numfunc', 'dia', 'hora_custom']]
hora_prim_saida.columns = ['numfunc', 'dia', 'hora_prim_saida']
hora_ult_entrada = intervalo_entrada.groupby(by=['numfunc', 'dia'], as_index=False).max().loc[:, ['numfunc', 'dia', 'hora_custom']]
hora_ult_entrada.columns = ['numfunc', 'dia', 'hora_ult_entrada']

# Cruzando dados
sample = sample.merge(hora_prim_saida, how='left', on=['numfunc', 'dia'])
sample = sample.merge(hora_ult_entrada, how='left', on=['numfunc', 'dia'])

sample.head()

In [ ]:
sample['dif_prim_saida'] = (sample['hora_prim_entrada'] - sample['hora_prim_saida']).apply(lambda x: x.total_seconds())
sample['dif_ult_entrada'] = (sample['hora_ult_saida'] - sample['hora_ult_entrada']).apply(lambda x: x.total_seconds())
#sample['dif_prim_saida'] = sample['hora_prim_entrada'] - sample['hora_prim_saida']
#sample['dif_ult_entrada'] = sample['hora_ult_saida'] - sample['hora_ult_entrada']

# Agrupando diferenças mínimas por funcional e dia
min_intervalo = sample.groupby(by=['numfunc', 'dia'], as_index=False).min().loc[:, ['numfunc', 'dia', 'dif_limite',
                                                                                    'hora_prim_saida', 'hora_ult_entrada',
                                                                                    'dif_prim_saida', 'dif_ult_entrada']]
# Aplicando subtração de primeira saída e última entrada
min_intervalo['flag_prim_saida'] = (min_intervalo['dif_prim_saida'] - min_intervalo['dif_limite'])
min_intervalo['flag_ult_entrada'] = (min_intervalo['dif_ult_entrada'] - min_intervalo['dif_limite'])
min_intervalo.head()

In [ ]:
# Filtrando elementos abaixo
min_intervalo['flag_intervalo_jornada_entrada'] = ((min_intervalo['dif_prim_saida'] < min_intervalo['dif_limite']) & (min_intervalo['flag_prim_saida'] > 0))
min_intervalo['flag_intervalo_jornada_saida'] = ((min_intervalo['dif_ult_entrada'] < min_intervalo['dif_limite']) & (min_intervalo['flag_ult_entrada'] > 0))

min_intervalo['flag_intervalo_jornada'] = ((min_intervalo['dif_prim_saida'] < min_intervalo['dif_limite']) & (min_intervalo['flag_prim_saida'] > 0)) | \
                                          ((min_intervalo['dif_ult_entrada'] < min_intervalo['dif_limite']) & (min_intervalo['flag_ult_entrada'] > 0))

min_intervalo['flag_intervalo_jornada_entrada'] = min_intervalo['flag_intervalo_jornada_entrada'] * 1
min_intervalo['flag_intervalo_jornada_saida'] = min_intervalo['flag_intervalo_jornada_saida'] * 1
min_intervalo['flag_intervalo_jornada'] = min_intervalo['flag_intervalo_jornada'] * 1
min_intervalo.head(15)

In [ ]:
min_intervalo['flag_intervalo_jornada'].value_counts()

In [ ]:
df_indicadores